In [54]:
import numpy as np

In [2]:
import urllib3
import urllib
import simplejson as json
import pprint
import requests
import os
from time import sleep

In [3]:
Lista_organizaciones = requests.get("http://comovamos.eokoe.com/api/3/action/organization_list").json()['result']
ciudad_verbose = {'barranquilla':"Barranquilla", 'bogota':"Bogotá", 'bucaramanga-metropolitana':"Bucaramanga Metropolitana", "cali": "Cali","cartagena":"Cartagena","ibague":"Ibagué","manizales":"Manizales","medellin":"Medellín", "pereira":"Pereira","valledupar":"Valledupar","yumbo":"Yumbo"}

In [4]:
APIkey = "ae464011-643c-47d5-8a44-1c93f7c7ea49"
Lista_organizaciones_loop = Lista_organizaciones

In [5]:
def generate_dataset_data(ciudad, dato):
    title_dict = {"objetivos_anual":"Síntesis de los indicadores recopilados anualmente para la ciudad de "+ ciudad_verbose[ciudad],"objetivos_lote":"Recopilacion de Indicadores para la ciudad de "+ ciudad_verbose[ciudad], "subjetivos_lote": "Recopilacion de los resultados de las encuestas de percepción ciudadana para "+ ciudad_verbose[ciudad],"subjetivos_archivo":"Archivo de todos los resultados de las encuestas para "+ ciudad_verbose[ciudad],"subjetivos_anual":"Resultados anuales de la encuesta de percepcion ciudadana de "+ ciudad_verbose[ciudad]}
    nombre_dict={"objetivos_anual": ciudad+"-objetivos_anual", "objetivos_lote": ciudad+"-objetivos_lote","subjetivos_lote":ciudad+"-subjetivos_lote","subjetivos_archivo":ciudad+"-subjetivos_archivo","subjetivos_anual":ciudad+"-subjetivos_anual"}
    tags_split = dato.split("_")
    dataset_dict = {
        "name": nombre_dict[dato],
        "title": title_dict[dato],
        "author":"Autor Ejemplo",
        "author_email":"example@msn.com",
        "mantainer":"Oscar",
        "mantainer_email":"mantainer@msn.com",
        "license_id":"odc-by",
        "tags": [{"name": ciudad}, {"name": tags_split[0]}, {"name": tags_split[1]}],
        #"owner_org": ciudad
        "owner_org": ciudad
    }
    return dataset_dict,nombre_dict[dato]

In [6]:
def generate_dataset(ciudad,dato):
    dataset_dict, nombre_dict = generate_dataset_data(ciudad,dato)
    d_url = "http://comovamos.eokoe.com/api/action/package_create"
    # Use the json module to dump the dictionary to a string for posting.
    data_string = urllib.parse.quote(json.dumps(dataset_dict))
    header = {'Authorization':APIkey,'content-type': 'application/x-www-form-urlencoded'}
    # We'll use the package_create function to create a new dataset.
    r = requests.post(d_url, data=data_string,headers=header)
    # Creating a dataset requires an authorization header.
    # Replace *** with your API key, from your user account on the CKAN site
    # that you're creating the dataset on.
    # Make the HTTP request.
    return nombre_dict

In [7]:
def upload_city(ciudad):
    #generar datasets
    tipos_datos=["objetivos_lote","objetivos_anual","subjetivos_lote","subjetivos_archivo","subjetivos_anual"]
    id_datasets = generate_city_datasets(ciudad,tipos_datos)
    print("Done with Datasets")
    #subir recursos en directorio
    generate_city_resources(ciudad,tipos_datos,id_datasets)
    return ciudad

In [8]:
def generate_city_datasets(ciudad,tipos_datos):
    id_datasets = []
    for dato in tipos_datos:
        id_datasets.append(generate_dataset(ciudad,dato))
        print(id_datasets)
    return id_datasets

In [26]:
def generate_city_resources(ciudad,tipos_datos,id_datasets):
    i = 0
    for tipo in tipos_datos:
        base_dir = tipo
        file_path = "data/"+ciudad+"/"+base_dir + "/"
        try:
            os.remove(file_path+".DS_Store")
        except OSError:
            pass
        files_inpath = os.listdir(file_path)
        for file in files_inpath:
            print(file)
            generate_resource(id_datasets[i],file,file_path)
            sleep(60)
        i = i + 1

In [10]:
def generate_resource(dataset_name, resource_name, file_path):
    full_filepath = file_path+resource_name
    r2 = requests.post('http://comovamos.eokoe.com/api/action/resource_create',
                  data={"package_id":dataset_name, "name":resource_name},
                  headers={"X-CKAN-API-Key": APIkey},
                  files=[('upload', open(full_filepath)),('url',"")])
    print(r2.text)
    return resource_name

In [11]:
Lista_organizaciones_loop.pop

['barranquilla',
 'bogota',
 'bucaramanga-metropolitana',
 'cali',
 'cartagena',
 'ibague',
 'manizales',
 'medellin',
 'pereira',
 'red-de-ciudades-como-vamos',
 'valledupar',
 'yumbo']

In [42]:
Lista_organizaciones_loop

['ibague', 'manizales', 'medellin', 'pereira', 'valledupar', 'yumbo']

In [43]:
for organizacion in Lista_organizaciones_loop:
    upload_city(organizacion)

['ibague-objetivos_lote']
['ibague-objetivos_lote', 'ibague-objetivos_anual']
['ibague-objetivos_lote', 'ibague-objetivos_anual', 'ibague-subjetivos_lote']
['ibague-objetivos_lote', 'ibague-objetivos_anual', 'ibague-subjetivos_lote', 'ibague-subjetivos_archivo']
['ibague-objetivos_lote', 'ibague-objetivos_anual', 'ibague-subjetivos_lote', 'ibague-subjetivos_archivo', 'ibague-subjetivos_anual']
Done with Datasets
diccionario_indicadores_ibague.csv
{"help": "http://comovamos.eokoe.com/api/3/action/help_show?name=resource_create", "success": true, "result": {"cache_last_updated": null, "package_id": "8d6ca87c-86d5-4e20-9866-9233ebf48e30", "webstore_last_updated": null, "id": "1135e32e-6374-41b2-87e6-c1d2c614f04c", "size": null, "state": "active", "hash": "", "description": "", "format": "CSV", "tracking_summary": {"total": 0, "recent": 0}, "mimetype_inner": null, "url_type": "upload", "mimetype": null, "cache_url": null, "name": "diccionario_indicadores_ibague.csv", "created": "2015-12-03

In [ ]:
r2 = requests.post('http://comovamos.eokoe.com/api/action/resource_create',
              data={"package_id":"barranquilla-objetivos", "name":"lol"},
              headers={"X-CKAN-API-Key": APIkey},
              files=[('upload', open('barranquilla/objetivos/indicadores_Barranquilla.csv')),('url',"")])

In [17]:
Lista_organizaciones_loop.pop(-4)

'red-de-ciudades-como-vamos'

In [23]:
upload_city("cali")

['cali-objetivos_lote']
['cali-objetivos_lote', 'cali-objetivos_anual']
['cali-objetivos_lote', 'cali-objetivos_anual', 'cali-subjetivos_lote']
['cali-objetivos_lote', 'cali-objetivos_anual', 'cali-subjetivos_lote', 'cali-subjetivos_archivo']
['cali-objetivos_lote', 'cali-objetivos_anual', 'cali-subjetivos_lote', 'cali-subjetivos_archivo', 'cali-subjetivos_anual']
Done with Datasets
diccionario_indicadores_cali.csv
{"help": "http://comovamos.eokoe.com/api/3/action/help_show?name=resource_create", "success": true, "result": {"cache_last_updated": null, "package_id": "ee113775-0a73-4e3d-be26-748b01735fd4", "webstore_last_updated": null, "id": "0ed551b4-5da7-4fc7-8b38-b8cadfeb9272", "size": null, "state": "active", "hash": "", "description": "", "format": "CSV", "tracking_summary": {"total": 0, "recent": 0}, "mimetype_inner": null, "url_type": "upload", "mimetype": null, "cache_url": null, "name": "diccionario_indicadores_cali.csv", "created": "2015-11-26T21:28:18.583116", "url": "http://

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable

In [57]:
a = np.nan

In [58]:
a is np.nan

True